In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
!ls

EDA and metrics.ipynb model                 server.py
README.md             mvnw                  src
data                  mvnw.cmd
dataset               pom.xml


In [13]:
raw = pd.read_csv('data/host.csv', names=['host'])

In [14]:
raw.drop_duplicates()

,host
0,api.youla.io
1,favicon.yandex.net
2,w-74721.fp.kaspersky-labs.com
3,questtime.net
4,passport-authproxy.taxi.yandex.net
...,...
999991,m39.hefxosi.com
999993,m42.ytoxcrj.com
999995,m5.fupppzz.net
999997,m5.ehgeqxn.me


In [15]:
def df_process(df):
    df['len'] = df['host'].map(lambda x: len(x))
    df['dots'] = df['host'].map(lambda x: x.count('.'))
    df['digits'] = df['host'].map(lambda x: sum([x.count(str(c)) for c in range(10)]))
    df['dash'] = df['host'].map(lambda x: x.count('-'))    
    return df

In [16]:
raw = df_process(raw)

In [17]:
raw.describe()

,len,dots,digits,dash
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,22.485400,2.297148,2.293751,0.712215
std,11.133069,1.093478,4.588589,1.218499
min,4.000000,1.000000,0.000000,0.000000
25%,15.000000,2.000000,0.000000,0.000000
50%,19.000000,2.000000,1.000000,0.000000
75%,27.000000,3.000000,2.000000,1.000000
max,84.000000,32.000000,41.000000,11.000000


In [18]:
len(dataset[dataset['y'] == 0])

NameError: name 'dataset' is not defined

In [ ]:
tech.index = range(len(tech))

In [ ]:
user.index = range(len(user))

In [ ]:
user

In [ ]:
tech = dataset[dataset['y'] == 0]

In [ ]:
user = pd.read_csv('dataset/user_sites.csv', sep=';', names=['host', 'y'])

In [ ]:
user = user[:len(tech)]

In [ ]:
dataset = pd.concat([user, tech])

In [ ]:
dataset = shuffle(dataset)

In [ ]:
dataset.to_csv('dataset/data.csv')

In [ ]:
dataset = pd.read_csv('dataset/data.csv', index_col=0)

In [ ]:
tech = dataset[dataset['y'] == 0]

In [ ]:
dataset

In [ ]:
len(dataset)

In [ ]:
dataset = df_process(dataset)

In [ ]:
dataset.describe()

In [ ]:
user = dataset[dataset['y'] == 1]
tech = dataset[dataset['y'] == 0]

In [ ]:
user.describe()

In [ ]:
user[user['dash'] > 2].head() 

In [ ]:
dataset

### редирект

In [ ]:
tech.describe()

In [ ]:
print(str(round(100 * len(user) / len(dataset), 2)) + '%')

In [19]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)

NameError: name 'dataset' is not defined

In [ ]:
charset = sorted(set([
    c for l in raw['host'] for c in l
]))

In [ ]:
charset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, KFold
from sklearn import metrics
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [20]:
def vectorize(hosts, vocab=charset):
    vectorizer= CountVectorizer(
                analyzer = "char",  
                tokenizer = None,  
                preprocessor = None, 
                stop_words = None,  
                max_features = 5000,
                vocabulary=vocab,
    )
    
    X = vectorizer.fit_transform(hosts)
    
    return X.toarray()
    
    

NameError: name 'charset' is not defined

In [21]:
X = vectorize(dataset['host'], vocab=charset)
y = dataset['y'].values

NameError: name 'vectorize' is not defined

In [22]:
from catboost import Pool, cv, CatBoostRegressor, CatBoost, CatBoostClassifier

In [23]:
params = {
    'iterations': 100,
    'loss_function': 'Logloss',
    'verbose': False,
}

cv_ds = Pool(data=X,
            label=y)
scores = cv(cv_ds,
           params,
           fold_count=2,
           plot='True')

NameError: name 'X' is not defined

In [24]:
X = dataset['host'].values

NameError: name 'dataset' is not defined

In [25]:
kf = KFold(n_splits=5, shuffle=True)

NameError: name 'KFold' is not defined

In [26]:
from model.Model import Model

In [27]:
metrics = {
    'accuracy': [],
    'roc_auc': [],
    'f1_score': [],
}

from tqdm import tqdm
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = Model(depth=8)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    metrics['accuracy'].append(accuracy_score(y_test, y_pred))
    metrics['roc_auc'].append(roc_auc_score(y_test, y_pred))
    metrics['f1_score'].append(f1_score(y_test, y_pred))

NameError: name 'kf' is not defined

In [278]:
for k, v in metrics.items():
    print(f'{k}: {np.round(np.mean(v), 4)}')

accuracy: 0.9502
roc_auc: 0.9502
precision: 0.9411
recall: 0.9599


In [283]:
print(str(round(100 * len(dataset[dataset['y'] == 1]) / len(dataset), 2)) + '%')

50.0%


In [208]:
from model.Model import Model

In [209]:
model = Model()

In [211]:
model.fit(dataset['host'], y)

Learning rate set to 0.129702
0:	learn: 0.5305536	total: 188ms	remaining: 3m 7s
1:	learn: 0.4287488	total: 316ms	remaining: 2m 37s
2:	learn: 0.3692330	total: 437ms	remaining: 2m 25s
3:	learn: 0.3279142	total: 555ms	remaining: 2m 18s
4:	learn: 0.3015253	total: 657ms	remaining: 2m 10s
5:	learn: 0.2813459	total: 779ms	remaining: 2m 9s
6:	learn: 0.2685742	total: 900ms	remaining: 2m 7s
7:	learn: 0.2564830	total: 1.01s	remaining: 2m 5s
8:	learn: 0.2488897	total: 1.13s	remaining: 2m 4s
9:	learn: 0.2422002	total: 1.25s	remaining: 2m 3s
10:	learn: 0.2363815	total: 1.36s	remaining: 2m 2s
11:	learn: 0.2329778	total: 1.47s	remaining: 2m
12:	learn: 0.2292937	total: 1.59s	remaining: 2m
13:	learn: 0.2261749	total: 1.73s	remaining: 2m 2s
14:	learn: 0.2236552	total: 1.83s	remaining: 2m
15:	learn: 0.2209117	total: 1.93s	remaining: 1m 58s
16:	learn: 0.2192877	total: 2.03s	remaining: 1m 57s
17:	learn: 0.2163612	total: 2.15s	remaining: 1m 57s
18:	learn: 0.2144905	total: 2.25s	remaining: 1m 56s
19:	learn: 0

158:	learn: 0.1606709	total: 20s	remaining: 1m 45s
159:	learn: 0.1605409	total: 20.2s	remaining: 1m 45s
160:	learn: 0.1604491	total: 20.3s	remaining: 1m 45s
161:	learn: 0.1602881	total: 20.5s	remaining: 1m 45s
162:	learn: 0.1601331	total: 20.6s	remaining: 1m 45s
163:	learn: 0.1599750	total: 20.8s	remaining: 1m 45s
164:	learn: 0.1598173	total: 20.9s	remaining: 1m 45s
165:	learn: 0.1596446	total: 21.1s	remaining: 1m 45s
166:	learn: 0.1594911	total: 21.4s	remaining: 1m 46s
167:	learn: 0.1592837	total: 21.6s	remaining: 1m 46s
168:	learn: 0.1590829	total: 21.7s	remaining: 1m 46s
169:	learn: 0.1590484	total: 21.8s	remaining: 1m 46s
170:	learn: 0.1589318	total: 22s	remaining: 1m 46s
171:	learn: 0.1587915	total: 22.2s	remaining: 1m 46s
172:	learn: 0.1586375	total: 22.3s	remaining: 1m 46s
173:	learn: 0.1585358	total: 22.5s	remaining: 1m 46s
174:	learn: 0.1583254	total: 22.6s	remaining: 1m 46s
175:	learn: 0.1580538	total: 22.7s	remaining: 1m 46s
176:	learn: 0.1579376	total: 23.1s	remaining: 1m 4

315:	learn: 0.1406697	total: 44.7s	remaining: 1m 36s
316:	learn: 0.1405710	total: 44.8s	remaining: 1m 36s
317:	learn: 0.1404452	total: 44.9s	remaining: 1m 36s
318:	learn: 0.1403471	total: 45s	remaining: 1m 36s
319:	learn: 0.1402282	total: 45.2s	remaining: 1m 35s
320:	learn: 0.1400848	total: 45.3s	remaining: 1m 35s
321:	learn: 0.1400181	total: 45.4s	remaining: 1m 35s
322:	learn: 0.1398538	total: 45.5s	remaining: 1m 35s
323:	learn: 0.1398159	total: 45.6s	remaining: 1m 35s
324:	learn: 0.1397053	total: 45.8s	remaining: 1m 35s
325:	learn: 0.1396047	total: 45.9s	remaining: 1m 34s
326:	learn: 0.1395253	total: 46s	remaining: 1m 34s
327:	learn: 0.1394571	total: 46.1s	remaining: 1m 34s
328:	learn: 0.1393823	total: 46.2s	remaining: 1m 34s
329:	learn: 0.1393012	total: 46.3s	remaining: 1m 34s
330:	learn: 0.1391776	total: 46.4s	remaining: 1m 33s
331:	learn: 0.1390822	total: 46.6s	remaining: 1m 33s
332:	learn: 0.1390143	total: 46.7s	remaining: 1m 33s
333:	learn: 0.1389400	total: 46.8s	remaining: 1m 3

471:	learn: 0.1282986	total: 1m 10s	remaining: 1m 19s
472:	learn: 0.1282038	total: 1m 10s	remaining: 1m 19s
473:	learn: 0.1281012	total: 1m 11s	remaining: 1m 19s
474:	learn: 0.1280661	total: 1m 11s	remaining: 1m 18s
475:	learn: 0.1279938	total: 1m 11s	remaining: 1m 18s
476:	learn: 0.1279445	total: 1m 11s	remaining: 1m 18s
477:	learn: 0.1278546	total: 1m 12s	remaining: 1m 18s
478:	learn: 0.1278052	total: 1m 12s	remaining: 1m 18s
479:	learn: 0.1277483	total: 1m 12s	remaining: 1m 18s
480:	learn: 0.1276658	total: 1m 12s	remaining: 1m 18s
481:	learn: 0.1276206	total: 1m 12s	remaining: 1m 18s
482:	learn: 0.1275474	total: 1m 13s	remaining: 1m 18s
483:	learn: 0.1274064	total: 1m 13s	remaining: 1m 18s
484:	learn: 0.1273101	total: 1m 13s	remaining: 1m 18s
485:	learn: 0.1271722	total: 1m 13s	remaining: 1m 17s
486:	learn: 0.1270928	total: 1m 13s	remaining: 1m 17s
487:	learn: 0.1270899	total: 1m 14s	remaining: 1m 17s
488:	learn: 0.1270629	total: 1m 14s	remaining: 1m 17s
489:	learn: 0.1270317	total:

625:	learn: 0.1194498	total: 1m 39s	remaining: 59.2s
626:	learn: 0.1194022	total: 1m 39s	remaining: 59s
627:	learn: 0.1193917	total: 1m 39s	remaining: 58.8s
628:	learn: 0.1192929	total: 1m 39s	remaining: 58.7s
629:	learn: 0.1192153	total: 1m 39s	remaining: 58.5s
630:	learn: 0.1191749	total: 1m 39s	remaining: 58.3s
631:	learn: 0.1191681	total: 1m 39s	remaining: 58.2s
632:	learn: 0.1191177	total: 1m 40s	remaining: 58s
633:	learn: 0.1190516	total: 1m 40s	remaining: 57.9s
634:	learn: 0.1189718	total: 1m 40s	remaining: 57.7s
635:	learn: 0.1188845	total: 1m 40s	remaining: 57.6s
636:	learn: 0.1188333	total: 1m 40s	remaining: 57.4s
637:	learn: 0.1187595	total: 1m 40s	remaining: 57.2s
638:	learn: 0.1187079	total: 1m 41s	remaining: 57.1s
639:	learn: 0.1186328	total: 1m 41s	remaining: 56.9s
640:	learn: 0.1184797	total: 1m 41s	remaining: 56.8s
641:	learn: 0.1183857	total: 1m 41s	remaining: 56.6s
642:	learn: 0.1183421	total: 1m 41s	remaining: 56.4s
643:	learn: 0.1182793	total: 1m 41s	remaining: 56.

781:	learn: 0.1100251	total: 2m 3s	remaining: 34.5s
782:	learn: 0.1100235	total: 2m 4s	remaining: 34.4s
783:	learn: 0.1100234	total: 2m 4s	remaining: 34.2s
784:	learn: 0.1099793	total: 2m 4s	remaining: 34.1s
785:	learn: 0.1099078	total: 2m 4s	remaining: 33.9s
786:	learn: 0.1098994	total: 2m 4s	remaining: 33.8s
787:	learn: 0.1098939	total: 2m 5s	remaining: 33.6s
788:	learn: 0.1098885	total: 2m 5s	remaining: 33.5s
789:	learn: 0.1098312	total: 2m 5s	remaining: 33.3s
790:	learn: 0.1097788	total: 2m 5s	remaining: 33.2s
791:	learn: 0.1097655	total: 2m 5s	remaining: 33s
792:	learn: 0.1097426	total: 2m 5s	remaining: 32.8s
793:	learn: 0.1096338	total: 2m 5s	remaining: 32.7s
794:	learn: 0.1095833	total: 2m 6s	remaining: 32.5s
795:	learn: 0.1095405	total: 2m 6s	remaining: 32.4s
796:	learn: 0.1095124	total: 2m 6s	remaining: 32.2s
797:	learn: 0.1094570	total: 2m 6s	remaining: 32s
798:	learn: 0.1094570	total: 2m 6s	remaining: 31.9s
799:	learn: 0.1094385	total: 2m 6s	remaining: 31.7s
800:	learn: 0.10

938:	learn: 0.1033122	total: 2m 27s	remaining: 9.61s
939:	learn: 0.1032837	total: 2m 28s	remaining: 9.45s
940:	learn: 0.1032113	total: 2m 28s	remaining: 9.29s
941:	learn: 0.1031710	total: 2m 28s	remaining: 9.13s
942:	learn: 0.1031269	total: 2m 28s	remaining: 8.98s
943:	learn: 0.1030661	total: 2m 28s	remaining: 8.82s
944:	learn: 0.1030316	total: 2m 28s	remaining: 8.66s
945:	learn: 0.1029606	total: 2m 28s	remaining: 8.5s
946:	learn: 0.1029365	total: 2m 29s	remaining: 8.35s
947:	learn: 0.1029096	total: 2m 29s	remaining: 8.19s
948:	learn: 0.1028919	total: 2m 29s	remaining: 8.03s
949:	learn: 0.1027920	total: 2m 29s	remaining: 7.87s
950:	learn: 0.1027079	total: 2m 29s	remaining: 7.71s
951:	learn: 0.1026678	total: 2m 29s	remaining: 7.56s
952:	learn: 0.1026507	total: 2m 30s	remaining: 7.4s
953:	learn: 0.1026426	total: 2m 30s	remaining: 7.24s
954:	learn: 0.1026375	total: 2m 30s	remaining: 7.08s
955:	learn: 0.1026300	total: 2m 30s	remaining: 6.93s
956:	learn: 0.1026271	total: 2m 30s	remaining: 6

CatBoostError: (No such file or directory) util/system/file.cpp:848: can't open "../saved/" with mode WrOnly|CreateAlways|Seq (0x00000034)

In [281]:
model.best_score_

{}

In [215]:
dataset['host'][:10]

455612                       s-usc1c-nss-201.firebaseio.com
67372                                           ptltest.com
118075                                 joycasino.sptpub.com
100562                                               awg.gl
34284     b8a1syx87gyqgjegt2ila6trefm6mmln7woctuinfa9e9f...
75020                                        ochoafoods.com
376375                           oxygen.stream.voidboost.in
77757                                     littlestraw.bc.ca
6878                                          tgirldiva.com
145489                                      mdap.alipay.com
Name: host, dtype: object

In [282]:
features = model.get_feature_importance()

CatBoostError: Model has no meta information needed to calculate feature importances.                         Pass training dataset to this function.

In [230]:
the_most_important = features.argsort()[-3:][::-1]

In [233]:
np.array(charset)[the_most_important]

array(['.', 'o', 'm'], dtype='<U1')

In [235]:
user.describe()

,y,len,dots,digits,dash
count,203995.0,203995.000000,203995.000000,203995.000000,203995.000000
mean,1.0,16.231648,1.266619,0.108836,0.123910
std,0.0,5.180874,0.518640,0.685331,0.415591
min,1.0,2.000000,0.000000,0.000000,0.000000
25%,1.0,13.000000,1.000000,0.000000,0.000000
50%,1.0,16.000000,1.000000,0.000000,0.000000
75%,1.0,19.000000,1.000000,0.000000,0.000000
max,1.0,67.000000,6.000000,36.000000,10.000000
